# CS145: Project 2
## Part 1 | Exploring World Bank Datasets with Colaboratory (40 points)
---



### Notes (read carefully!):

* Be sure you read the instructions on each cell and understand what it is doing before running it.
* Don't forget that if you can always re-download the starter notebook from the course website if you need to.
* You may create new cells to use for testing, debugging, exploring, etc., and this is in fact encouraged!
**Just make sure that the final answer for each question is _in its own cell_ and _clearly indicated_**.
* Colab will not warn you about how many bytes your SQL query will consume.  **Be sure to check on the BigQuery UI first before running queries here!**
* See the assignment handout for submission instructions.
* Have fun!

## Collaborators:
Please list the names and SUNet IDs of your collaborators below:
* *Hang Jiang, hjiang42*

## Setting up BigQuery and Dependencies



Run the two cells below (shift + enter) to authenticate your project and import the libraries you'll need. 

Note that you need to fill in the `project_id` variable with the Google Cloud project id you are using for this course.  You can see your project ID by going to https://console.cloud.google.com/cloud-resource-manager

In [0]:
# Run this cell to authenticate yourself to BigQuery.
from google.colab import auth
auth.authenticate_user() 
project_id = "rapid-snowfall-218803"

In [0]:

# Some imports you will need
import pandas as pd
import altair as alt

### How to BigQuery in Collab

Jupyter notebooks (what Collab notebooks are based on) have a concept called "magics".
If you write the following line at the top of a `Code` cell:

```
%%bigquery --project $project_id variable # this is the key line
SELECT ....
FROM ...
```

That "%%" converts the cell into a SQL cell. The resulting table that is generated is saved into `variable`. Then in a second cell:

```python
alt.Chart(variable).mark_line().encode(
...
)
```

You can use the variable like so in order to create a chart! 

# Section 1 | Schema Design!

---





The World Bank collects and aggregates data from many public sources around the world and publishes it online. Google BigQuery has made this data available for us to play with, and it contains a tremendous number of metrics about nation-level activity and outcomes. 

For this project, we will be using the [`world_bank_health_population`](https://bigquery.cloud.google.com/dataset/bigquery-public-data:world_bank_health_population) public dataset.

## Question 1: Describe the dataset of the World Bank (1 point)

If you had to describe the way the data is set up in the World Bank datasets (any of the four datasets, as they have identical structure), what would you say about it? 
**Note:** The rest of this question is about going through the structure of the dataset, so we only need a couple of sentences here. We're looking for your impressions - what do you notice?

---
- Some tables can be decomposed furthre. For example, in `country_series_definition`, country_code can deduce description but not series_code, so this table can be decomposed to country_code, series_code and country_code, description according to BCNF.
- In the schema interface, it lacks description. 
---

## Meet OKV, the Anti-Schema

---

**`OKV`** stands for `Object-Key-Value` [[1]](https://colab.research.google.com/drive/1asMZjcxwBqGpurcPOZ6pmtdUaXipWdmX#scrollTo=ZvK6dAU2k8Qa&line=4&uniqifier=1). It's a way of storing information in a database that has the opposite of a schema - you're free to define any property you'd like on any object. Think of it like a gigantic hashmap (10B rows is nothing with this kind of model [[2]](https://colab.research.google.com/drive/1asMZjcxwBqGpurcPOZ6pmtdUaXipWdmX#scrollTo=ZvK6dAU2k8Qa&line=4&uniqifier=1)) for every variable on every object in your system. 



Here's one way a table for such a system could be laid out:


|  object  |             key          |  value  
|-------------|---------------------------|------
|    102    |        "name"         |  "John Watson"
|    103    |        "name"         |  "Sherlock Holmes"
|    102    |       "address"      |  "221B Baker Street, London, UK"
|    107    |        "name"         |  "Oprah Winfrey"
|    103    |       "address"      |  "221B Baker Street, London, UK"
|    102    |        "canes"        |  26
|    103    |  "cases_solved" |  60




As you can see, the three objects in this table all have different "shapes" (another word for "schema" in systems that don't have a formal schema). 

When you want to query for an object you will need a query like :

```sql
SELECT key, value
FROM table
WHERE object = 102
```

Then merging all the rows in the output will give you the object!


### Notes
1. Other three word versions of this idea include ID-Key-Value, Object-Property-Value, Entity-Attribute-Value, Entity-Property-Value and they have their respective three-letter-acronym (TLA) as well
2. The reason being that the number of rows in an OKV store = number of cells in a regular table.


### Further Reading (Optional)
* Wikipedia [article ](https://en.wikipedia.org/wiki/Entity%E2%80%93attribute%E2%80%93value_model) on this structure of data
* This (OKV store, and database design in general) is a great thing to ask Shiva about during his office hours!

## Question 2: Ruminating about OKVs (6 points)

Compare and contrast the OKV store to the traditional relational table (the kind you see in class) - 
What are the advantages? What are some of the difficulties? 
#### (Keep your answer around 200 words, bullet-points suggested!)

### Hints
* One helpful checklist is the acronym **CRUD**, which stands for **Create**, **Read**, **Update**, **Delete**. These are the four basic data operations. You can create/read/update/delete the values in a database, or the schema of a database (eg, add/delete a property, or change its type). 

* When you think about advantages and disadvantages in software, some of the common desirable attributes are - performance (how fast queries run), memory footprint (less is better), maintainability (can the database gracefully change with product requirements?), and code complexity (does a db design encourage large, unwieldy queries? Programmers are human after all - the more complexity, the more bugs!). Combining these attributes with the operations above should give you a place to start your comparison.

* An extra hint: We haven't talked about performance in databases yet, but for the purposes of this question, you can think in three tiers:
  * Lookup: You have a key of a table, and you want the row (or some subset of the row). You can conceptualize this as O(1)
  * Scan: When you need to lookup rows by some criteria (eg, people taller than 6 feet). Height is not a key, so the database has to scan all the rows. You can conceptualize this as O(N). Scanning also happens when writing a property to many rows at once.
  * Join: When you join tables together, it creates a Cartesian Product of sets (also called pairs). If there are N rows in a table, and another table has roughly the same number of rows (so N there too), then you can think of a two-table join as O(N^2).


---
The form of OKV and classical tables
- The OKV make the relations from a tuple into colomns, so the table have more rows with less colomns(only three).
- The OKV makes it hard to detect the key(it become object in OKV), and attributes become key and tuples become value. So it is more difficult to find relationships between attibutes in OKV but easier to find the value if only one attribute is needed.

CURD 
- The OKV is easy to create/read/update/delete the values in a database, because they are just object key value. That is to say, when we only operate on a specific value, for example, to update on object 102's name, it is easy to find it and update it. 
- However, it is more difficult to operate on schemas, becase schemas are not show in one row. To be more specific, when we want to spot the relations between attributes, as they are listed in keys without logic, it's not easy. And if we want to add/delete something in the schema, we have to change lots of times in the keys while in classical table we only have to change once. 

- The OKV will consume more memory because it will store the keys for many times while the classical table only have to store it once in the schema. When running the table, OKV have to find the object for many times while the classical one only have to find once. The OKV is easier to maintain if we just want to change a specific data. But if we want to change a tuple or schema, it is more difficult. db design may not encourage large, unwieldy queries like OKV. Using classical one shows more logic. 



---



---

## One More Thing - Property Names




As you learned in class, redundant data in a table is undesirable, because if you want to change something, then you have to update all the rows corresponding to that value (which is very expensive; remember that an OKV model has way more rows than a traditional table). This is also called an **update anomaly**.

How to solve this issue? Simple, a property table:

```sql
# Schema (in the syntax of the Gradiance homework): 
Property(id, name)
Data(id, key, value)
```

So we'd take the table above, and replace it with:

Property Table:

  
| id | name |
|-----|----------|
| 1  | "name"
| 2  | "address"
| 3  | "canes"
| 4  | "cases_solved"


Data Table:

|    id    |  pid   |  value
--------|-------------|-------------
|  102   |  1      |  "John Watson"
|  103   |  1      |  "Sherlock Holmes"
|  102   |  2      |  "221B Baker Street, London, UK"
|  107   |  1      |  "Oprah Winfrey"
|  103   |  2      |  "221B Baker Street, London, UK"
|  102   |  3      |  26
|  103   |  4      |  60


## Question 3: Rumination Redux (2 points)
Update your comparison above to account for the property table - which things have been made easy to do with this change, and which things are still hard to do? 

Please only comment on the differences - you don't need to redo the whole analysis here.

---
- It is easier to read by people as we can see all attributes.
- To select all attributes of a objects, it is still difficult because we have to select many times. 
- It is easier to modify the schema because we only have to change it in the property table and don't have to change all things in the key colomn.

---

## One MORE Thing - Types! 

In `SQL`, every column is required to have a type [[1]](https://colab.research.google.com/drive/1asMZjcxwBqGpurcPOZ6pmtdUaXipWdmX#scrollTo=inVjXmRsiXc2&line=3&uniqifier=1). So when we mixed *string* values and *int* values in a single column, that was a simplification. 

There are multiple design options that work for this situation - see the next question for a discussion of them.


#### Notes
1. There are databases that do not offer this feature/suffer from this curse (depending on your opinions), where you are free to put whatever value you'd like in the third column as shown above. If you're curious why anyone would want that - consider the difference between a static vs dynamically typed programming language (e.g. Java vs Python). There are similar tradeoffs in choosing a database that has typed schema vs type agnostic schema. This is another great thing to talk to Shiva about if you have more questions. 

## Question 4:  A Friendly Dialogue (6 points)



(PS - **This is a very common software engineering interview question!**).

Your well meaning friend tries to implement an OKV store in SQL and runs into the snag above. Let's say, for simplification, they are only interested in storing* string* values and* int *values (you can imagine extending this to more types).

They propose the following solution (assume the property table also exists, it's not shown here):

id    | pid | string_value       | int_value
------|-------|---------------------------|--------
102 | 1   | "Sherlock Holmes" | null
103 | 1   | "John Watson"     | null
102 | 3   | null                        | 60


To explain: if the value is a *string* value, then fill the `string_value` column and fill the `int_value` column with null; vice versa if there's an *int* value. 


### a) What's wrong with this picture? (2 points)

If you had to critique this design, what would you tell your friend? Again, what is difficult or not desirable about this design?

---
- There are null in each row, it wastes space and causes redundency.

---

### b) Your alternative (2 points)

Come up with an alternative design (hint: you may need more than 1 table) that addresses some of the concerns you brought up above.


---
Property(id, name)  
Data1(id, pid, string_value)  
Data2(id, pid, int_value)

---


### c)  No free lunch! (2 points)

Your friend looks at your design - what critique would he offer you? What is made difficult or not desirable about your design?

---
- You have to seperate string_value and int_value. Another thing is that you have to repeat id and pid twice. 

---

## Apply Your Knowledge!

The world bank data is structured as a OKV... with a small twist. The tables that contain the information are more or less arranged as follows:

```
object | key | year | value
```

Where `object` = country code & `key` = indicator code.

There are a couple of other fields (a description for the object and the key), but the overall structure is still OKV.

With this new understanding try reading the schema and identifying other features of the key-value store setup that we identified above. 


## Question 5: Schema Comprehension (3 points)

Each of the following parts is worth 1 point.


### a)  What is the name of the table that functions as the property table in the four world bank data sets?


---
- health_nutrition_population

---

### b) Which table contains extra information about the "objects" (as in Object-Key-Value) in the table?


---
- country_series_definitions

---


### c) What is the "key" (as in Object-Key-Value) of the `health_nutrition_population` table?

---
indicator_name   
indicator_code

---

## Question 6: Design Theory (12 points)

Design your own schema for the world bank data! Your goal is to make visualizations to answer these questions:

* What is the breakdown of population by decade (0-9, 10-19, 20-29, etc) by male and female for the US over time? 
* Is the US population shifting towards an aging one or a young one? 
* How does the population breakdown of the US and China (or any other country) differ? 
* What is the life expectancy versus the health care expenditure for all the countries of the world?
* Where is HIV in the world? Make a picture that shows the distribution of AIDS patients by major region.  

#### Further Requirements

* No matter which schema you choose, it should be clear how you could add data from more indicators to it.  This could be adding a column(s), row(s) or table(s) depending on your design.
  * Eg, if you're not storing GDP/capita, how could you add it to the table?
* 



#### Hints:

In the real world, stuff happens! We want our databases to be able to handle those cases.
Remember the CRUD (create, read, update, delete) acronym! What if:
* A statistic turns out to be incorrect and it needs to be updated?
* You need to add all the data for all the countries when it's published at the end of 2018?
* A country splits into two because of a revolutionary war?
* A country changes its name back to a pre-colonial era one?
* You need to store very small percentages (think prevalence of rare diseases)?
* There are statistics that only apply to some countries and not others (eg, fishing in a landlocked country)?
* You suddenly want to store data at a higher sampling rate (say, monthly or weekly rather than yearly)?

It's very unlikely your design performs well under all these conditions (and many more you can come up with) and that's okay! No design is perfect - we're looking for you to show your understanding of the tradeoffs you made and what that means for any application you write on top of your database.

### a) What entities are present in your schema? (2 points)


---

there are two entities sets:  
Countries and Series   

---

### b) What is the relationship between them? (You don't need to draw a perfect ER diagram - you can also just list 'one to one', 'one to many' and 'many to many' for each pair). (2 points)



---
The relationship between countries and series is a country do investigation on the content of series.  
many to many

---

### c) Draw out your tables (like you've seen above and in class), and clearly note which column(s) form the key for that table, and which columns are keys of another table (foreign keys). (3 points)


---


*Example table for formatting reference. The values are intended to be samples, we just need a couple rows to get an idea of what's stored where:*

table `investigation`:   

country_code | year | value | series_code
----------------|-----------------|-----------------|----------------
   AUS  |   2000   |  26652.0  |  SP.DYN.LE00.IN
  CHN  |   2000   |   96568.0  |  SH.DYN.AIDS

keys:   country_code and series_code and year  
foreign key: series_code and country_code are respectively foreign keys

table `coutries`:  

country_code | short_name |  long_name
----------------|-----------------|-----------------|----------------
  GMB  |   The Gambia   |   Republic of The Gambia  
   CHN  |   China   |   People's Republic of China 

keys: country_code  

table `series`:  

series_code | topic | indicator_name
----------------|-----------------|-----------------|----------------
   SP.DYN.LE00.IN  |   Health: Mortality   |  Life expectancy at birth, total (years) 
   SH.DYN.AIDS  |   Health: Risk factors   |  Adults (ages 15+) living with HIV  
   
Keys: indicator_name

---

### d) List the (minimal) functional dependencies that are present in your tables. (2 points)


---
table `investigation`  
{country_code,country_code ,series_code, year} -> {value}  

table `countries`  
{country_code} -> {country_code, short_name ,long_name}

table `sesries`  
{series_code} -> {topics, indicator_name}

---

### e) Comment on your design - what is it good/bad for? *What are the tradeoffs that you're making in choosing your design?* (3 points)


---

GOOD  
- One which minimizes the possibility of anomalies
- Reduce redundencies   

BAD
- I give up some data
- To make the table easier to understand, I choose topic and long_name. Though they can be deduced by the key and make the table bigger. 


---

# Section 2 | Learn you a Visualization for Great Good
---

(This title is a pun on a famous series of programming books that look like `Learn you a ____ for Great Good`.)

In this section, you'll be answering questions, similar to the first project. The difference is that instead of just answering with a query, you will be answering with a visualization. Part of this assignment is for you to think about what type of chart/picture/visualization will convey your information well, and to think about which data (specifically, which indicators) you should be using in order to answer a particular question. 

We're focusing on visualizations because they are a primary method of understanding and communicating the nature of data. Especially with the large datasets that are available today, a picture is worth 1M rows :) 

For a look at what is possible, see [Gapminder](https://www.gapminder.org/tools). Gapminder is a professional visualization of all the world bank indicators which is also interactive! You can look up some cool TED talks that use Gapminder to display world metrics (it's a very popular tool).

If you need to see "the answers" for some of the relational (see: scatterplot) type data, feel free to look it up here and verify that you have something that looks right. Also, part of this project will be choosing the right indicators. You are free to use Gapminder to play with different options before deciding on one!

## General Instructions
* For each question, you will have at least two cells - a SQL cell where you run your query (and save the output to a data frame), and a visualization cell, where you construct your chart. Please be clear that **all** data manipulation is to be done in SQL. Please do not use `pandas` or any other python library to massage your data output - in the real world, that would be impossible. 
* Please make all charts legible - this inclues axes labels, clear tick marks, clear point markers/lines/color schemes (i.e. don't repeat colors across categories), legends, and so on. 
* If you're asking for help, be sure to talk to the TAs about which indicators you are using and how you determined they were the right ones. Ultimately we care about the chart, so if you're aggregating the same data stored in a different way, that's fine (eg population by decade instead of population by phase of life). Some indicators will lead to easier solutions though, so we encourage you to spend some time making sure you've got one that feels straightforward to use. Feel free to come to office hours if you want to discuss finding the right indicators. 

## Visualization Libraries
The Colab notebook comes preinstalled with a visualization library called **Altair**. 
You can read its docs here: https://altair-viz.github.io/ 

There are some basic code snippets available if you open the menu on the left of the notebook and look under code snippets. 
We expect you to read the docs and understand how to use this library. The exploration you do now will help for project 3 when you do your own visualizations as well, and for later in your career if you ever want to play around with data (the Jupyter notebook, which is what Colab is based on, is a common data analysis tool these days!)





## Indicators

The World Bank indicators are available and searchable [here](https://data.worldbank.org/). If you want to browse some of what's available, you can check [here](https://data.worldbank.org/indicators). The browsing page doesn't have all the indicators (despite saying "all indicators", it only has the "Primary World Bank" indicators).  
If that fails, then you can Google "World Bank ____________ indicator" and click on the results from the world bank that come up.

You will likely need to look up indicator codes and indicator code patterns in order to extract the necessary data for this portion of the assignment. Once you've found your indicator, you will arrive at a page like `https://data.worldbank.org/indicator/XXXXXXXXX`. The X's are where the `indicator_code` for that indicator are found. For example, in `https://data.worldbank.org/indicator/SH.XPD.CHEX.GD.ZS`, the `SH.XPD.CHEX.GD.ZS` is the `indicator_code` you would search for in the database.

Alternately, you are free to query for keywords in BigQuery directly (it may be easier for some of the simpler plots).

Many of these questions are _intentionally_ left open ended for you to think about what the right metrics are (another important data skill). Part of asking and answering interesting questions is thinking about the blind spots of your metrics. 
For example, say you plot $ spent versus educational attainment for various countries. Would you rather use the raw money spent, or money as % GDP (Gross Domestic Product)? What are the tradeoffs?

## Question 7 (3 points)


First, something basic - let's plot the total population of the USA over time as a stacked area chart - there should be multiple bands for the granularities of population that are recorded (0-14, 15-64, 65+). The x axis should be 'year' and the y axis should be 'population'. The sum of all the bands will equal the total population in the US at that time. 

 **Hint:** BigQuery's REGEX functions may be helpful here. (You may want to test your regex at regex101.com before using it in BigQuery to make sure it works!)

In [20]:
%%bigquery --project $project_id q7 

SELECT country_name, indicator_name, year, value
FROM  `bigquery-public-data.world_bank_health_population.health_nutrition_population`
WHERE regexp_contains(indicator_name, r"Population ages") 
AND regexp_contains(indicator_name, r"total")
AND NOT regexp_contains(indicator_name, r"%")
AND country_name = "United States"


,country_name,indicator_name,year,value
0,United States,"Population ages 00-14, total",1960,55442894.0
1,United States,"Population ages 00-14, total",1961,56573837.0
2,United States,"Population ages 00-14, total",1962,57357660.0
3,United States,"Population ages 00-14, total",1963,57864629.0
4,United States,"Population ages 00-14, total",1964,58220136.0
5,United States,"Population ages 00-14, total",1965,58419941.0
6,United States,"Population ages 00-14, total",1966,58471461.0
7,United States,"Population ages 00-14, total",1967,58412472.0
8,United States,"Population ages 00-14, total",1968,58217334.0
9,United States,"Population ages 00-14, total",1969,57911781.0


In [27]:
# YOUR PLOT CODE HERE

alt.Chart(q7).mark_area().encode(
    x="year:N",
    y="value:Q",
    color="indicator_name:N"
   
)


Chart({
  data:       country_name                       indicator_name  year       value
  0    United States         Population ages 00-14, total  1960  55442894.0
  1    United States         Population ages 00-14, total  1961  56573837.0
  2    United States         Population ages 00-14, total  1962  57357660.0
  3    United States         Population ages 00-14, total  1963  57864629.0
  4    United States         Population ages 00-14, total  1964  58220136.0
  5    United States         Population ages 00-14, total  1965  58419941.0
  6    United States         Population ages 00-14, total  1966  58471461.0
  7    United States         Population ages 00-14, total  1967  58412472.0
  8    United States         Population ages 00-14, total  1968  58217334.0
  9    United States         Population ages 00-14, total  1969  57911781.0
  10   United States         Population ages 00-14, total  1970  57616587.0
  11   United States         Population ages 00-14, total  1971  57056877.0
  12   United States         Population ages 00-14, total  1972  56399805.0
  13   United States         Population ages 00-14, total  1973  55654422.0
  14   United States         Population ages 00-14, total  1974  54833306.0
  15   United States         Population ages 00-14, total  1975  54006943.0
  16   United States         Population ages 00-14, total  1976  53532331.0
  17   United States         Population ages 00-14, total  1977  52989186.0
  18   United States         Population ages 00-14, total  1978  52437806.0
  19   United States         Population ages 00-14, total  1979  51972624.0
  20   United States         Population ages 00-14, total  1980  51556408.0
  21   United States         Population ages 00-14, total  1981  51325205.0
  22   United States         Population ages 00-14, total  1982  51210012.0
  23   United States         Population ages 00-14, total  1983  51201638.0
  24   United States         Population ages 00-14, total  1984  51263141.0
  25   United States         Population ages 00-14, total  1985  51411147.0
  26   United States         Population ages 00-14, total  1986  51854058.0
  27   United States         Population ages 00-14, total  1987  52332807.0
  28   United States         Population ages 00-14, total  1988  52860890.0
  29   United States         Population ages 00-14, total  1989  53437830.0
  ..             ...                                  ...   ...         ...
  144  United States  Population ages 65 and above, total  1988  30468750.0
  145  United States  Population ages 65 and above, total  1989  30976879.0
  146  United States  Population ages 65 and above, total  1990  31512637.0
  147  United States  Population ages 65 and above, total  1991  32043729.0
  148  United States  Population ages 65 and above, total  1992  32550223.0
  149  United States  Population ages 65 and above, total  1993  32998803.0
  150  United States  Population ages 65 and above, total  1994  33374786.0
  151  United States  Population ages 65 and above, total  1995  33690562.0
  152  United States  Population ages 65 and above, total  1996  33946497.0
  153  United States  Population ages 65 and above, total  1997  34178513.0
  154  United States  Population ages 65 and above, total  1998  34375571.0
  155  United States  Population ages 65 and above, total  1999  34565009.0
  156  United States  Population ages 65 and above, total  2000  34763336.0
  157  United States  Population ages 65 and above, total  2001  35047948.0
  158  United States  Population ages 65 and above, total  2002  35306397.0
  159  United States  Population ages 65 and above, total  2003  35568227.0
  160  United States  Population ages 65 and above, total  2004  35912118.0
  161  United States  Population ages 65 and above, total  2005  36330941.0
  162  United States  Population ages 65 and above, total  2006  36893128.0
  163  United States  Population ages 65 and above, total  2007  37524268.0
  164  United States  Population a

## Question 8 (3 points)

Is the US population aging or getting younger overall? Make a normalized, stacked area chart so you can see the answer!

In [0]:
# As Shown in the graph, the proportion of population under 14 is smaller while the 14~64 and over 65 is bigger
# So population get aging 

In [31]:
%%bigquery --project $project_id q8
SELECT indicator_name, year, value
FROM `bigquery-public-data.world_bank_health_population.health_nutrition_population` 
WHERE regexp_contains(indicator_name, r"Population ages") AND
regexp_contains(indicator_name, r"(% of total)")  AND
NOT regexp_contains(indicator_name, r"male") AND 
NOT regexp_contains(indicator_name, r"female")
AND country_name = "United States"


,indicator_name,year,value
0,Population ages 00-14 (% of total),1960,30.687213
1,Population ages 00-14 (% of total),1961,30.798372
2,Population ages 00-14 (% of total),1962,30.748512
3,Population ages 00-14 (% of total),1963,30.577054
4,Population ages 00-14 (% of total),1964,30.340528
5,Population ages 00-14 (% of total),1965,30.066412
6,Population ages 00-14 (% of total),1966,29.747386
7,Population ages 00-14 (% of total),1967,29.395543
8,Population ages 00-14 (% of total),1968,29.006275
9,Population ages 00-14 (% of total),1969,28.573435


In [36]:
# YOUR PLOT CODE HERE
alt.Chart(q8).mark_area().encode(
    alt.Y("value:Q", axis=alt.Axis(title="population percentage")),
    x="year:N",
    color="indicator_name:N"
   
)


Chart({
  data:                                 indicator_name  year      value
  0           Population ages 00-14 (% of total)  1960  30.687213
  1           Population ages 00-14 (% of total)  1961  30.798372
  2           Population ages 00-14 (% of total)  1962  30.748512
  3           Population ages 00-14 (% of total)  1963  30.577054
  4           Population ages 00-14 (% of total)  1964  30.340528
  5           Population ages 00-14 (% of total)  1965  30.066412
  6           Population ages 00-14 (% of total)  1966  29.747386
  7           Population ages 00-14 (% of total)  1967  29.395543
  8           Population ages 00-14 (% of total)  1968  29.006275
  9           Population ages 00-14 (% of total)  1969  28.573435
  10          Population ages 00-14 (% of total)  1970  28.098525
  11          Population ages 00-14 (% of total)  1971  27.475971
  12          Population ages 00-14 (% of total)  1972  26.870357
  13          Population ages 00-14 (% of total)  1973  26.263359
  14          Population ages 00-14 (% of total)  1974  25.640533
  15          Population ages 00-14 (% of total)  1975  25.006340
  16          Population ages 00-14 (% of total)  1976  24.552173
  17          Population ages 00-14 (% of total)  1977  24.059856
  18          Population ages 00-14 (% of total)  1978  23.558553
  19          Population ages 00-14 (% of total)  1979  23.093299
  20          Population ages 00-14 (% of total)  1980  22.689584
  21          Population ages 00-14 (% of total)  1981  22.367237
  22          Population ages 00-14 (% of total)  1982  22.105295
  23          Population ages 00-14 (% of total)  1983  21.900509
  24          Population ages 00-14 (% of total)  1984  21.737789
  25          Population ages 00-14 (% of total)  1985  21.608222
  26          Population ages 00-14 (% of total)  1986  21.593891
  27          Population ages 00-14 (% of total)  1987  21.599333
  28          Population ages 00-14 (% of total)  1988  21.620085
  29          Population ages 00-14 (% of total)  1989  21.650615
  ..                                         ...   ...        ...
  144  Population ages 65 and above (% of total)  1988  12.461707
  145  Population ages 65 and above (% of total)  1989  12.550443
  146  Population ages 65 and above (% of total)  1990  12.624092
  147  Population ages 65 and above (% of total)  1991  12.666457
  148  Population ages 65 and above (% of total)  1992  12.689453
  149  Population ages 65 and above (% of total)  1993  12.695802
  150  Population ages 65 and above (% of total)  1994  12.683956
  151  Population ages 65 and above (% of total)  1995  12.652402
  152  Population ages 65 and above (% of total)  1996  12.601059
  153  Population ages 65 and above (% of total)  1997  12.535351
  154  Population ages 65 and above (% of total)  1998  12.461509
  155  Population ages 65 and above (% of total)  1999  12.387116
  156  Population ages 65 and above (% of total)  2000  12.320329
  157  Population ages 65 and above (% of total)  2001  12.298865
  158  Population ages 65 and above (% of total)  2002  12.275141
  159  Population ages 65 and above (% of total)  2003  12.260343
  160  Population ages 65 and above (% of total)  2004  12.264846
  161  Population ages 65 and above (% of total)  2005  12.294044
  162  Population ages 65 and above (% of total)  2006  12.364481
  163  Population ages 65 and above (% of total)  2007  12.456966
  164  Population ages 65 and above (% of total)  2008  12.581766
  165  Population ages 65 and above (% of total)  2009  12.750891
  166  Population ages 65 and above (% of total)  2010  12.971848
  167  Population ages 65 and above (% of total)  2011  13.242522
  168  Population ages 65 and above (% of total)  2012  13.556765
  169  Population ages 65 and above (% of total)  2013  13.903462
  170  Population ages 65 and above (% of total)  2014  14.267328
  171  Population ages 65 and above (% of total)  2015  14.640343
  172  Population ages 65 

## Question 9 (4 points)

Let's make a plot just like Gapminder - Who's getting the most health for their money? Plot "money spent on healthcare" versus "life expectancy" (play with Gapminder to find the right metrics; there are a few options). Make a bubble plot where the size is population of the country, the bubbles are colored by region, and use a slider to change the year (note: choose something reasonable for the range)! Please also include some way of seeing which country is which (a tooltip, perhaps).

In [9]:
%%bigquery --project $project_id q9
SELECT c1.country_code, expectancy, expenditure, population, c1.year
FROM 
(SELECT country_code, year, value AS expectancy
FROM `bigquery-public-data.world_bank_health_population.health_nutrition_population` 
WHERE indicator_name = "Life expectancy at birth, total (years)" )c1,

(SELECT country_code,year, value AS expenditure
FROM `bigquery-public-data.world_bank_health_population.health_nutrition_population` 
WHERE indicator_name = "Current health expenditure per capita (current US$)" )c2,

(SELECT country_code, indicator_name, year, value AS population
FROM  `bigquery-public-data.world_bank_health_population.health_nutrition_population`
WHERE indicator_code = "SP.POP.TOTL")c3,

(SELECT country_code
FROM `bigquery-public-data.world_bank_health_population.country_summary`
WHERE currency_unit != "") c4

WHERE c1.year = c2.year AND c1.year = c3.year AND c2.year = c3.year 
AND c1.country_code = c2.country_code AND c1.country_code = c3.country_code AND c2.country_code = c3.country_code
AND c1.country_code = c4.country_code







,country_code,expectancy,expenditure,population,year
0,ZMB,44.702000,24.453997,10531221.0,2000
1,HND,70.537000,68.949536,6524283.0,2000
2,GIN,51.181000,16.954525,8808546.0,2000
3,NIC,69.742000,53.010833,5026796.0,2000
4,LSO,48.405000,28.544302,1868699.0,2000
5,PER,70.512000,90.223769,25914879.0,2000
6,LVA,70.314634,258.777923,2367550.0,2000
7,ERI,55.258000,11.752986,3392801.0,2000
8,ZAF,56.341000,221.782986,45728315.0,2000
9,DOM,70.568000,115.399187,8562622.0,2000


In [12]:
# YOUR PLOT CODE HERE
    slider = alt.binding_range(min=2000, max=2015, step=1)
select_year = alt.selection_single(name="year", fields=['year'], bind=slider)
alt.Chart(q9).mark_circle().encode(
    alt.X('expenditure:Q', scale=alt.Scale(type='log')),
    alt.Y('expectancy:Q', scale=alt.Scale(zero=False)),
    size='population:Q',
    color=alt.Color('country_code', legend=None),
    tooltip = ['country_code']
).add_selection(
    select_year).transform_filter(
    select_year
).interactive()

Chart({
  data:      country_code  expectancy  expenditure   population  year
  0             ZMB   44.702000    24.453997   10531221.0  2000
  1             HND   70.537000    68.949536    6524283.0  2000
  2             GIN   51.181000    16.954525    8808546.0  2000
  3             NIC   69.742000    53.010833    5026796.0  2000
  4             LSO   48.405000    28.544302    1868699.0  2000
  5             PER   70.512000    90.223769   25914879.0  2000
  6             LVA   70.314634   258.777923    2367550.0  2000
  7             ERI   55.258000    11.752986    3392801.0  2000
  8             ZAF   56.341000   221.782986   45728315.0  2000
  9             DOM   70.568000   115.399187    8562622.0  2000
  10            MKD   73.267000    71.563267    2034819.0  2000
  11            SLE   38.702000    13.782684    4564297.0  2000
  12            BHR   74.440000   466.708291     664614.0  2000
  13            STP   63.338000    53.911637     138606.0  2000
  14            UZB   67.154000    29.266546   24650400.0  2000
  15            BLZ   68.329000   132.615056     247315.0  2000
  16            LBY   70.473000   244.772793    5355751.0  2000
  17            SLV   68.834000   178.794627    5867626.0  2000
  18            GMB   55.924000    23.007984    1231844.0  2000
  19            MYS   72.800000   105.765963   23185608.0  2000
  20            NGA   46.266000    14.621436  122352009.0  2000
  21            COL   71.019000   135.334451   40403958.0  2000
  22            CRI   77.448000   249.128670    3925443.0  2000
  23            DNK   76.592683  2496.047241    5339616.0  2000
  24            KGZ   68.558537    12.290353    4898400.0  2000
  25            COD   50.041000    18.444592   47076387.0  2000
  26            KEN   51.751000    21.203801   31450483.0  2000
  27            COM   59.460000    45.550984     542357.0  2000
  28            NAM   55.608000   207.456113    1899257.0  2000
  29            CYP   78.010000   751.077332     943286.0  2000
  ...           ...         ...          ...          ...   ...
  2816          DOM   73.673000   396.692522   10528394.0  2015
  2817          THA   75.103000   217.135232   68657600.0  2015
  2818          CAF   51.410000    16.639663    4546100.0  2015
  2819          SLB   70.430000   152.129097     587482.0  2015
  2820          IRQ   69.672000   154.477493   36115649.0  2015
  2821          DEU   80.641463  4591.846096   81686611.0  2015
  2822          HUN   75.568293   893.654457    9843028.0  2015
  2823          UGA   59.575000    46.058465   40144870.0  2015
  2824          JAM   75.836000   294.261106    2871934.0  2015
  2825          BLR   73.624390   351.824926    9489616.0  2015
  2826          VUT   71.925000    98.986144     264603.0  2015
  2827          URY   77.341000  1281.291902    3431552.0  2015
  2828          IRL   81.453659  4757.117548    4701957.0  2015
  2829          TUR   75.498000   454.608292   78271472.0  2015
  2830          NGA   52.985000    97.306861  181181744.0  2015
  2831          MEX   76.933000   534.810930  125890949.0  2015
  2832          LAO   66.335000    52.985258    6663967.0  2015
  2833          ISR   82.051220  2756.117125    8380100.0  2015
  2834          SGP   82.743902  2280.284125    5535002.0  2015
  2835          GHA   62.448000    79.588280   27582821.0  2015
  2836          MDA   71.468000   186.434761    3554108.0  2015
  2837          KAZ   72.000000   379.076365   17542806.0  2015
  2838          LVA   74.480488   783.845657    1977527.0  2015
  2839          BRA   75.284000   780.395982  205962108.0  2015
  2840          GRD   73.531000   460.275620     106823.0  2015
  2841          TCD   52.575000    35.569097   14009413.0  2015
  2842          MNE   76.938000   382.084214     622159.0  2015
  2843          TZA   64.950000    31.737349   53879957.0  2015
  2844          CRI   79.634000   929.114974    4807852.0  2015
  2845          LSO   53.745000    90.850119    2174645.0  2015
  
  [2846 rows x 